In [8]:
import pandas as pd
import numpy as np

import matplotlib as m_plt
from matplotlib import pyplot as plt
import matplotlib.mlab as mlabQ

import seaborn as sns

from copy import copy
from pprint import pprint

import sys
import os

import personal_plotter as p_plt

In [9]:
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

### 1.4 Hypotesis Related Questions and Answers

#### 1.4.1 the more close residential center is from the Industrial zone, the denser are?

Using the parameters ***Available & Rooms*** as variables in the formula:

$$ Available = av \land Rooms = ro $$

$$ \frac{av - \min{(av)}}{\max{(av)}} \cdot {ro} = \rho_{people} $$

the data is almost conclusive , but even if the Weak Hypothesis has been reinforced , the strong hypothesis has just been "supported".
    
 - Strong Hypothesis: + Near to, + People

 - Weak Hypothesis: - Near To, - People

#### 1.4.2 [1.4.1] being one of the closers,  implies more accessibility?

Yes, but not only. The more closest is, it is more probably have better accessibility.

In both cases, the values are enclosed almost in the confidence interval.

This hypotesis has been reinforced

#### 1.4.3 rent increased the more nearly you are?

yes, but this only happens in the left skew, if we check the right one; the expected behavior fails. It is possible to say this hypothesis remains inconclusive (still, it's more close to reach some conclusion).

#### 1.5 Issues

* The data isn't normalize
* Have NaN values
* Small and not so detailed dictionary of variables 

_ _ _
___
_ _ _


## 2.- Data 2 - Instituições de Educação Superior
### 2.1.- Retrieving Data

#### 2.1.1.- Reading Data

In [14]:
#/agro/data_dict.xlsx
try :
    agro_dataset = pd.read_csv("/agro/falhas_plantio_hashing.csv", sep=';', encoding='latin1')
except:
    print("ERROR, DATA NOT LOADED")
else:
    print("""DATA SUCCESSFULLY LOADED\n
    \tRows\t:\t{:d}\t|\tColumns\t:\t{:d}""".format(agro_dataset.shape[0], agro_dataset.shape[1]))

ERROR, DATA NOT LOADED


#### 2.1.2.- Data Labels

In [10]:
ieducationals_descriptions = pd.read_excel("./microdados_censo_superior_2014/2014/ANEXOS/ANEXO I/ANEXO I - 2014.xlsx", encoding='latin1')
ieducationals_descriptions.columns = ieducationals_descriptions.iloc[0,:]
ieducationals_descriptions = ieducationals_descriptions.reindex(ieducationals_descriptions.index.drop(0))
ieducationals_descriptions

,N,Nome da Variável,Descrição da Variável,Tipo,Tam.(1),Categorias
1,DADOS DA IES,NaN,NaN,NaN,NaN,NaN
2,1,CO_IES,Código único de identificação da IES,Num,8,NaN
3,2,NO_IES,Nome da IES,Char,200,NaN
4,3,SGL_IES,Sigla da IES,Char,20,NaN
5,4,CO_MANTENEDORA,Código único de identificação da mantenedora,Num,8,NaN
6,5,NO_MANTENEDORA,Nome da mantenedora,Char,200,NaN
7,6,CO_CATEGORIA_ADMINISTRATIVA,Código da Categoria Administrativa,Num,8,1. Pública Federal
8,NaN,NaN,NaN,NaN,NaN,2. Pública Estadual
9,NaN,NaN,NaN,NaN,NaN,3. Pública Municipal
10,NaN,NaN,NaN,NaN,NaN,4. Privada com fins lucrativos


#### 2.1.3.- Viewing Data

In [11]:
ieducations_dataset.iloc[10:15, :]

,CO_IES,NO_IES,SGL_IES,CO_MANTENEDORA,NO_MANTENEDORA,CO_CATEGORIA_ADMINISTRATIVA,DS_CATEGORIA_ADMINISTRATIVA,CO_ORGANIZACAO_ACADEMICA,DS_ORGANIZACAO_ACADEMICA,CO_MUNICIPIO_IES,...,VL_RECEITA_PROPRIA,VL_TRANSFERENCIA,VL_OUTRA_RECEITA,VL_DES_PESSOAL_REM_DOCENTE,VL_DES_PESSOAL_REM_TECNICO,VL_DES_PESSOAL_ENCARGO,VL_DES_CUSTEIO,VL_DES_INVESTIMENTO,VL_DES_PESQUISA,VL_DES_OUTRAS
10,18,UNIVERSIDADE CATÓLICA DE PELOTAS,UCPEL,18,SOCIEDADE PELOTENSE DE ASSISTENCIA E CULTURA(S...,5,Privada sem fins lucrativos,1,Universidade,4314407,...,7.894191e+07,0.00,5.797977e+06,2.602745e+07,1.253955e+07,3834444.75,1662554.20,1475652.23,3644347.33,4.160242e+07
11,19,PONTIFÍCIA UNIVERSIDADE CATÓLICA DE CAMPINAS,PUC-CAMPINAS,19,SOCIEDADE CAMPINEIRA DE EDUCACAO E INSTRUCAO,5,Privada sem fins lucrativos,1,Universidade,3509502,...,2.270624e+08,3923447.18,2.807421e+07,7.926935e+07,4.056702e+07,49420223.26,34381020.74,18843000.00,6963160.34,1.942829e+07
12,20,UNIVERSIDADE DE PASSO FUNDO,UPF,20,FUNDACAO UNIVERSIDADE DE PASSO FUNDO,5,Privada sem fins lucrativos,1,Universidade,4314100,...,2.639809e+08,2787447.31,8.246577e+07,9.031341e+07,3.436398e+07,48785819.70,96088925.43,23906071.76,5461299.75,6.577762e+07
13,21,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO RIO GRANDE...,PUCRS,21,UNIAO BRASILEIRA DE EDUCACAO E ASSISTENCIA,5,Privada sem fins lucrativos,1,Universidade,4314902,...,3.909450e+08,8001545.13,1.549417e+08,1.110816e+08,6.536671e+07,96889748.59,99739237.39,26216083.04,55019625.58,1.214524e+08
14,22,UNIVERSIDADE PRESBITERIANA MACKENZIE,MACKENZIE,22,INSTITUTO PRESBITERIANO MACKENZIE,5,Privada sem fins lucrativos,1,Universidade,3550308,...,5.542484e+08,0.00,3.880826e+07,1.977172e+08,1.213924e+08,81814354.01,0.00,36429948.69,2789691.71,1.314069e+08


In [12]:
print(np.sum(ieducations_dataset.isna().astype(int)))

print(np.sum(ieducations_dataset.isnull().astype(int)))

CO_IES                           0
NO_IES                           0
SGL_IES                        191
CO_MANTENEDORA                   0
NO_MANTENEDORA                   0
CO_CATEGORIA_ADMINISTRATIVA      0
DS_CATEGORIA_ADMINISTRATIVA      0
CO_ORGANIZACAO_ACADEMICA         0
DS_ORGANIZACAO_ACADEMICA         0
CO_MUNICIPIO_IES                 0
NO_MUNICIPIO_IES                 0
CO_UF_IES                        0
SGL_UF_IES                       0
NO_REGIAO_IES                    0
IN_CAPITAL_IES                   0
QT_TEC_TOTAL                     0
QT_TEC_FUND_INCOMP_FEM           0
QT_TEC_FUND_INCOMP_MASC          0
QT_TEC_FUND_COMP_FEM             0
QT_TEC_FUND_COMP_MASC            0
QT_TEC_MEDIO_FEM                 0
QT_TEC_MEDIO_MASC                0
QT_TEC_SUPERIOR_FEM              0
QT_TEC_SUPERIOR_MASC             0
QT_TEC_ESPECIALIZACAO_FEM        0
QT_TEC_ESPECIALIZACAO_MASC       0
QT_TEC_MESTRADO_FEM              0
QT_TEC_MESTRADO_MASC             0
QT_TEC_DOUTORADO_FEM

### 2.2.- Attributes Choose

#### 2.2.1.- Choosed Attributes and Hypothesis

**Hypothesis**: Private universities have the highest self-capital, but lower scientific investment (in comparison with national universities)

**Attributes**

* **DS_CATEGORIA_ADMINISTRATIVA (id 6)**        :  *Nome da Categoria Administrativa    *
    > Item Class.
* **DS_ORGANIZACAO_ACADEMICA (id 8)**        :  *Código da Organização Acadêmica*
    > Item Sub-Class
* **VL_RECEITA_PROPRIA (id 33)**       :  *Informa o valor das receitas próprias auferidas pela mantenedora ou pela IES no ano de referência*
    > Avaliable Expendable Money
* **VL_DES_INVESTIMENTO (id 40)**       :  *Informa o valor das despesas de investimentos (despesas de capital) realizados nas IES ou na mantenedora*
    > Expended Money I
* **VL_DES_PESQUISA (id 41)**     :  *Informa o valor das despesas com Pesquisa e Desenvolvimento da IES ou da mantenedora*
    > Expended Money I

#### 2.2.2.- Selecting Data

In [ ]:
case_ieducations_dataset = ieducations_dataset.iloc[:,[6, 8, 33, 40, 41]].copy()
case_ieducations_dataset.columns

#### 2.2.3.- Qualitative Data to Quantitave

In [ ]:
cat_admin_labels = np.unique(case_ieducations_dataset.iloc[:, 0]).copy()
cat_organ_labels = np.unique(case_ieducations_dataset.iloc[:, 1]).copy()

print("\n\t\tDS_CATEGORIA_ADMINISTRATIVA\t:\n", pd.DataFrame(cat_admin_labels), "\n", 
      "\n\t\tDS_ORGANIZACAO_ACADEMICA\t:\n", pd.DataFrame(cat_organ_labels), "\n\n")

# admin
ii = 0
labels = cat_admin_labels
temp_array = np.zeros(case_ieducations_dataset.shape[0], dtype=np.uint)
for dummy_category in labels:
    temp_whereis = (case_ieducations_dataset.iloc[:, ii] == dummy_category).astype(np.uint)
    temp_quantiv = np.where(labels == dummy_category)[0]
    temp_array +=  temp_whereis * np.uint(temp_quantiv)
    
case_ieducations_dataset.iloc[:, ii] = temp_array.T.copy()

# organ
ii = 1
labels = cat_organ_labels
temp_array = np.zeros(case_ieducations_dataset.shape[0], dtype=np.uint)
for dummy_category in labels:
    temp_whereis = (case_ieducations_dataset.iloc[:, ii] == dummy_category).astype(np.uint)
    temp_quantiv = np.where(labels == dummy_category)[0]    
    temp_array +=  temp_whereis * np.uint(temp_quantiv)
    
case_ieducations_dataset.iloc[:, ii] = temp_array.T.copy()

In [ ]:
case_ieducations_dataset = case_ieducations_dataset.sort_values(["DS_CATEGORIA_ADMINISTRATIVA", "DS_ORGANIZACAO_ACADEMICA"], ascending=[1, 0])
case_ieducations_dataset = case_ieducations_dataset.reset_index(drop=[0])

print(case_ieducations_dataset.iloc[10:15, :])

##### 2.2.4.- Creating Targets

Higest Self Capital + Lower Cientific Investmen = > 
both money , H * L = Ratio

In [ ]:
case_ieducations_dataset_norm = (case_ieducations_dataset - case_ieducations_dataset.min())
case_ieducations_dataset_norm = (case_ieducations_dataset_norm / case_ieducations_dataset_norm.max())

# normalized

temp_ratio = case_ieducations_dataset_norm["VL_RECEITA_PROPRIA"] * case_ieducations_dataset_norm["VL_DES_PESQUISA"]

temp_targets = p_plt.make_targets(temp_ratio, samples=3)
case_ieducations_dataset_norm["target"] = temp_targets - 1

### 2.3 Data Exploration

In [ ]:
p_plt.dummy_DataFrame_plotter(case_ieducations_dataset_norm,
                        plot_size=(15,15), 
                        mixed_style=True,
                        plot_style=["c--", "g."], 
                        grid=True).show()

#### 1.3.1 Histogram

 - It have outliers?
 
 > Yes , except in the 4th column (Accesibility)

***Below : 1rst Not Normalize Data, 2nd Normalize Data (normal base)***

##### PLOT

##### Type A

In [ ]:
in_normal_base_case_ieducations_dataset_norm = p_plt.to_normal_base(case_ieducations_dataset_norm)

p_plt.skew_demo(in_normal_base_case_ieducations_dataset_norm, 
                plot_size=(15,20), 
                grid_subplot=[3,2],
                grid=True)

In [ ]:
data_val = case_ieducations_dataset_norm

target_names = data_val['target'].unique()
columns = data_val.columns.tolist()

##### Type B

In [ ]:
plt.figure(figsize=(10, 10))
for col in range(1,6):
    ax = plt.subplot(3, 2, col)
    ax.set_title(columns[col-1])
    for c in data_val.target.unique():
        data_val.loc[ data_val.target == c, columns[col-1]].plot.hist(alpha=0.3)
    
plt.legend(target_names)
plt.tight_layout()
plt.show()

- the outliers are really Outliers?
    * Available: No, it has a Right Skew
    * Rooms: Apparently yes, but, could be just a distribution error
    * Distance: considering the max value is in the range of acceptable values, its possible affirm, indeed, it has real outliers.
    * Accessibility: Being a discrete set of values, doesn't show significant outliers, (even its possible it doesn't have any)
    * Cost x 1K: Similar to the *Available* set of values, it has a Right Skew

***See Below for referential data (the main box in boxplot are values between 25% and 75 Quartile)***

In [ ]:
sns.pairplot(data=data_val, vars=columns[:-1], hue='target')
plt.show()

In [ ]:
stats = data_val.describe()
print(stats)

In [ ]:
cov = data_val.iloc[:, :-1].cov()

sns.heatmap(cov)
plt.show()

In [ ]:
cor = data_val.iloc[:, :-1].corr()

sns.heatmap(cor)
plt.show()

In [ ]:
# covarianca por classe

plt.figure(figsize=(20, 6))
for c_i, c in enumerate(data_val.target.unique()):
    ax = plt.subplot(1, 4, c+1)
    plt.title(target_names[c_i])
    cov = data_val.loc[ data_val.target == c, columns[:-1]].cov()
    sns.heatmap(cov)
    
#plt.legend(target_names)
plt.tight_layout()
plt.show()

In [ ]:
# correlacao por classe

plt.figure(figsize=(20, 6))
for c_i, c in enumerate(data_val.target.unique()):
    ax = plt.subplot(1, 4, c+1)
    plt.title(target_names[c_i])
    cor = data_val.loc[ data_val.target == c, columns[:-1]].corr()
    sns.heatmap(cor)
    
#plt.legend(target_names)
plt.tight_layout()
plt.show()

#### 2.3.6 Confidence interval

it is possible to see in the plot above the Confidence Interval ( $[μ - 2σ, μ + 2σ]$ )

### 2.4 Hypotesis Related Questions and Answers

#### 2.4.1 are private universities expending less money in research?

in a general way, yes, even if some institutions are in the "higher research budget"  group the amount is not significative enough.

this hypothesis has been reinforced

#### 2.4.2 have public universities less self-capital?

No, in correlation whit other institutions, they self-capital is enough to be considered almost equal to mean (see Kurtosis).

this hypothesis has failed, one time.

#### 2.4.3 are private universities earning more money for his investment?

apparently yes, but still not conclusive.

this hypothesis remains equal.

#### 2.5 Issues

* The data isn't normalize
* Categorical Variables
* Excessive Heterogene Data Set